In [15]:
import pandas as pd
import numpy as np

In [16]:
def get_data_with_pd(file_name, sheet):
    data = pd.read_excel(file_name, sheet_name=sheet)
    return data

In [17]:
def load_all_pricing_matrices():
    service_types = [
        "FHD Ground",
        "FHD Ground Canada",
        "FedEx First Overnight",
        "FedEx Priority Overnight",
        "FedEx Standard Overnight",
        "FedEx 2Day A.M.",
        "FedEx 2Day",
        "FedEx Express Saver",
        "Intra-Hawaii Standard List Rate"
    ]
    
    mw_service_types = [
#         ADDRESS STRING FORMATTING AT SOME POINT
        "MW FedEx First Overnight",
        "MW FedEx Priority Overnight",
        "MW FedEx Standard Overnight",
        "MW FedEx 2Day A.M.",
        "MW FedEx 2Day",
        "MW FedEx Express Saver",
        "MWIntra-HawaiiFedExPriorityON"
    ]
    
    freight_service_types = [
        "FedEx First Overnight Freight",
        "FedEx 1Day Freight",
        "FedEx 2Day Freight",
        "FedEx 3Day Freight"
    ]
    
    pricing_matrices = {}
    all_sheets = pd.read_excel("./separated_us_express_rates.xlsx", sheet_name=None)
    
    for service_type in service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in mw_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in freight_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]

    return pricing_matrices

In [18]:
def dim_check(l, w, h, dim):
    if np.isnan(l) or np.isnan(w) or np.isnan(h) or np.isnan(dim):
#         IF ANY VALUE IS NOT A NUMBER (nan), CANNOT RUN MATH OPS
        return None
    elif dim == 0:
#         ANY NUM/0 == UNDEFINED
        return None
    else:    
        return l*w*h/dim

In [19]:
def add_accessorials(row):
#     ALL POSSIBLE ACCESSORIAL CHARGES
    accessorial_list = [
        row['OFSC$'],
        row['Late Fee_LF1-1'],
        row['OTP$'],
        row['ODS$'],
        row['Adult Signature'],
        row['Print Return Label'],
        row['ORES$'],
        row['ODASC$'],
        row['ODASEC$'],
        row['ODAS$'],
        row['ODASER$'],
        row['Address Correction'],
        row['Return On Call Surcharge'],
        row['ONDOCAC$']
    ]
#     SKIP OVER NAN VALUES TO AVOID ERRORS
    total = sum(n for n in accessorial_list if not np.isnan(n))
#     FORMAT FLOAT TO HUNDREDTHS PLACE 
    return float("{:.2f}".format(total))

In [23]:
def process_data(data):
#     MAY NEED TO SEPARATE PRICING FOR ZONES LISTED AS 9-10 AND 11-12 FOR SIMPLICITY OF CALC.

    pricing_matrices = load_all_pricing_matrices()
    
    for index, row in data.iterrows():
        weight = row['RW']
        zone = int(row['Z'])
        service_type = row['Service Type']
        otpub = row['OTPUB']
        omin = row['OMIN']
        svc_pkging = row['Service Packaging']
        l = row['L']
        w = row['W']
        h = row['H']
        dim = row['DIM']
        this_dim = dim_check(l, w, h, dim)
        
#         FEDEX STANDARD OVERNIGHT AND FEDEX 2DAY AM HAVE NO DATA FOR ZONES 10-12
#         FEDEX EXPRESS SAVER HAS NO DATA IN ZONE COLUMNS 9-10/11-12/13-16
        if svc_pkging == "FedEx Pak":
            pass
#             otpub.color = (255, 0, 0)

#         IF SERVICE TYPE HAS A MATCHING SHEET IN RATES BOOK
        elif service_type in pricing_matrices:
#             GRAB CORRESPONDING DATA
            pricing_matrix_data = pricing_matrices[service_type]
#             DETERMINE THE LARGER WEIGHT FOR CALC - DIM OR RW
            actual_weight = weight if this_dim == None or weight > this_dim else this_dim
#             IF ITS A FEDEX ENVELOPE UP TO 8oz
            if actual_weight == 0.5:
#                 USE INDEX 0 FOR FEDEX ENV ROW COORDINATE THEN ADD ACCESSORIAL CHARGES
                temp_otpub = pricing_matrix_data.iloc[0, zone-1] 
                new_otpub = float("{:.2f}".format(temp_otpub + add_accessorials(row)))
#                 CHECK TO SEE IF PRICE EXCEEDS MIN
                otpub = new_otpub if new_otpub > omin else omin
#                 print(otpub)
#             IF ITS NOT AN ENVELOPE
            else:
#                 TO GET ACCURATE COORDINATES, ADD 1 TO ACTUAL WEIGHT AND SUBTRACT 1 FROM ZONE
                temp_otpub = pricing_matrix_data.iloc[int(actual_weight)+1, zone-1] 
#                 ADD ACCESSORIAL CHARGES
                new_otpub = float("{:.2f}".format(temp_otpub + add_accessorials(row)))
#                 CHECK TO SEE IF PRICE EXCEEDS MIN
                otpub = new_otpub if new_otpub > omin else omin
#                 if new_otpub > omin:
#                     print("larger than min")
#                 else: 
#                     print("smaller than min")

        elif service_type == "Ground":
#             IF BEING SHIPPED OUTSIDE US
            if row["Recipient Country/Territory"] != "US":
#                 MANUALLY GRAB CORRESPONDING DATA
                pricing_matrix_data = pricing_matrices["FHD Ground Canada"]
#                 DETERMINE THE LARGER WEIGHT FOR CALC - DIM OR RW
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = int()
#                 ASSIGN CORRECT INDEX FOR DIFFERENT ZONING LAYOUT
                if zone == 51:
                    zone_index = 1
                elif zone == 54:
                    zone_index = 2
#                 TO GET ACCURATE COORDINATES, SUBTRACT 1 FROM WEIGHT AND USE MANUALLY ASSIGNED ZONE INDEX
                temp_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index]
#                 ADD ACCESSORIAL CHARGES
                new_otpub = float("{:.2f}".format(temp_otpub + add_accessorials(row)))
#                 CHECK TO SEE IF PRICE EXCEEDS MIN
                otpub = new_otpub if new_otpub > omin else omin
#                 print(otpub)
                
            else:
#                 MANUALLY GRAB CORRESPONDING DATA
                pricing_matrix_data = pricing_matrices["FHD Ground"]
#                 DETERMINE THE LARGER WEIGHT FOR CALC - DIM OR RW
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = zone
#                 ASSIGN CORRECT INDEX FOR DIFFERENT ZONING LAYOUT
                if zone == 14:
                    zone_index = 10
                elif zone == 17:
                    zone_index = 11
                elif zone == 22:
                    zone_index = 12
                elif zone == 23:
                    zone_index = 13
                elif zone == 25:
                    zone_index = 14
                elif zone == 92:
                    zone_index = 15
                elif zone == 96:
                    zone_index = 16
                    
#                 TO GET ACCURATE COORDINATES, SUBTRACT 1 FROM WEIGHT AND USE MANUALLY ASSIGNED ZONE INDEX
                temp_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index-1]
#                 ADD ACCESSORIAL CHARGES
                new_otpub = float("{:.2f}".format(new_otpub + add_accessorials(row)))
#                 CHECK TO SEE IF PRICE EXCEEDS MIN
                otpub = new_otpub if new_otpub > omin else omin
#                 print(otpub)

        else:
#             BELOW PRINTS ANY SERVICE TYPE FOR WHICH WE MAY HAVE NOT HAVE ANY HANDLING 
            print(f'{service_type} not addressed!')
#             otpub.color = (255, 0, 0) #CHANGES CELL COLOR TO RED FOR EASY RECOGNITION
    
#     AT THIS POINT, THE DATA MANIPULATED IS SIMPLY AN INSTANCE AND LATER WE MUST USE save_results_with_xlwings() FOR
#     MANIPULATED DATA TO PERSIST IN A WORKBOOK
    return data

In [24]:
def save_results_with_xlwings(data, file_name, sheet_name):
    print(data)
#     wb = xw.Book(file_name)
#     ws = wb.sheets[sheet_name]
#     ws.range('A1').options(index=False).value = data
#     wb.save()
#     wb.close()

In [25]:
#BEGIN MAIN SCRIPT USING ABOVE FUNCTIONS
file_name = './charges.xlsm'
spreadsheet_name = 'FREIGHT & ACCESSORIALS'
charge_data = get_data_with_pd(file_name, spreadsheet_name)
processed_data = process_data(charge_data)
# save_results_with_xlwings(processed_data, file_name, spreadsheet_name)

larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger than min
larger t